**NOTE** `phimats` environment should be used as kernel

In [1]:
from importlib import reload

import numpy as np

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

import BoundaryConditions
from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

### Simulation data

In [2]:
# Simulation name
Simul = "CrackTip"
# Simulation type
SimulType = "Mechanical"
# Element sets
nElementSets = 1
# Number of steps to achieve the load
nSteps = 500

### Read mesh file

In [3]:
# Path to .inp mesh file
modelName = "../cracktip"

mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 31977
  Number of cells:
    quad: 9872
    quad: 5856
    quad: 7920
    quad: 7920
  Point sets: Crack
  Cell sets: Crack


### Element data

In [4]:
# Element name
elementName = "quad"  		# meshio compatible element name
nNodes = mesh.points.shape[0]            # Total number of nodes
nodCoord = mesh.points[:,0:2]            # Node coordinates
nElements = mesh.cells_dict[elementName].data.shape[0]  # Total number of elements
elemNodeConn = mesh.cells_dict[elementName]        # Node connectivity for all elements

### Material data

In [5]:
# Analysis type
AnalysisType = "PlaneStrain"
# Isotropy
Isotropy = "Isotropic"
# Young's modulus
Emod = 200e9
# Poisson's ratio
nu = 0.3

# Plasticity type
Plasticity = "IsoHard"
HardeningLaw = "KME"
sig_y0 = 250e6    # Pa
rho_0 = 1e10  # Initial dislocation density (m⁻²)
M = 3 		  # Taylor factor
alpha = 0.3   # Dislocation interaction constant
b = 2.5e-10   # Burgers vector (m)
G = Emod/(2*(1+nu)) # Shear modulus
k1 = 2*(G/(200*G*alpha*b))  # Multiplication coefficient
k2 = 10  # Recovery coefficient

Materials = {
	"Material_1" : {
		"Elastic" : {
			"AnalysisType" : AnalysisType,
			"Isotropy" 	 : Isotropy,
			"Emod" 		 : Emod,
			"nu"   		 : nu,
		},
		"Plastic" : {
			"Plasticity"   : Plasticity,
			"HardeningLaw" : HardeningLaw,
			"sig_y0"       : sig_y0,
			"rho_0"        : rho_0,
			"M"            : M,
			"b"            : b,
			"alpha"        : alpha,
			"k1"           : k1,
   			"k2"           : k2,
		},
	},
}

Materials["Material_1"]["Plastic"]

{'Plasticity': 'IsoHard',
 'HardeningLaw': 'KME',
 'sig_y0': 250000000.0,
 'rho_0': 10000000000.0,
 'M': 3,
 'b': 2.5e-10,
 'alpha': 0.3,
 'k1': 133333333.33333333,
 'k2': 10}

### Boundary layer load

The boundary layer method is used to apply the load, with the crack-tip displacement fields [Anderson, T. Fracture Mechanics: Fundamentals and Applications. CRC Press (2017)]

$$u_x = \frac{K_I}{2\mu} \sqrt{\frac{r}{2\pi}} \cos\left(\frac{\theta}{2}\right) \left[ \alpha - 1 + 2\sin^2\left(\frac{\theta}{2}\right) \right]$$

$$u_y = \frac{K_I}{2\mu} \sqrt{\frac{r}{2\pi}} \sin\left(\frac{\theta}{2}\right) \left[ \alpha + 1 - 2\cos^2\left(\frac{\theta}{2}\right) \right]$$

Where:
- $K_I$: Mode I Stress intensity factor
- $\mu$: Shear modulus
  - $\mu = \frac{E}{2(1+\nu)}$, where $E$ is Young's modulus and $\nu$ is Poisson's ratio.
- $\alpha$: Constant depending on plane strain or plane stress:
  - **Plane strain:** $\alpha = 3 - 4\nu$

In [6]:
# Parameters for boundary layer model
K_I = 89.2*1e6     # Stress intensity factor (e.g., MPa*sqrt(m))
mu = Emod / (2 * (1 + nu))	# Shear modulus
kappa = 3 - 4 * nu  	# For plane strain

### Apply boundary conditions
**NOTE** This is the total load to be achieved in number of steps `nSteps`.

In [7]:
meshBC = meshio.read("../cracktip_BC.inp")
# print(meshBC)

# Compute displacements
displacements = []

for iNod in range(len(meshBC.points[meshBC.point_sets["Load"]])):
    
	x = meshBC.points[meshBC.point_sets["Load"]][iNod,0]
	y = meshBC.points[meshBC.point_sets["Load"]][iNod,1]
	
	# Convert to polar coordinates
	r = np.sqrt(x**2 + y**2)
	theta = np.arctan2(y, x)  
 
	# Avoid division by zero at the crack tip
	if r == 0:
		displacements.append((x, y, 0.0, 0.0))
		continue

	# Mode I displacement fields
	u_x = (K_I/(2*mu))*np.sqrt(r/(2*np.pi))*np.cos(theta/2)*(kappa - 1 + 2*np.sin(theta/2)**2)
	u_y = (K_I/(2*mu))*np.sqrt(r/(2*np.pi))*np.sin(theta/2)*(kappa + 1 - 2*np.cos(theta/2)**2)
	
	displacements.append((meshBC.point_sets["Load"][iNod], u_x, u_y))

# Fixed
fixed = []   
for iNod in range(len(meshBC.points[meshBC.point_sets["Fix"]])):
	if (meshBC.point_sets["Fix"][iNod]!=6):
		fixed.append((meshBC.point_sets["Fix"][iNod], 0, 0))

In [8]:
presBCs = [] 

for iNod in range(len(displacements)):
	presBCs.append([displacements[iNod][0], 0, displacements[iNod][1]])
	presBCs.append([displacements[iNod][0], 1, displacements[iNod][2]])

# y-fixed
for iNod in range(len(fixed)):
	# presBCs.append([fixed[iNod][0], 0, fixed[iNod][1]])
	presBCs.append([fixed[iNod][0], 1, fixed[iNod][2]])


In [9]:
# Write mesh into for inspection 
BCmesh = WriteDispBCs("CrackTip", "quad", mesh, presBCs)

### Initialize simulation object

In [11]:
SimulationData = {
    "Simul"            : Simul,
	"SimulType"		   : SimulType,
    "mesh"             : mesh,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : presBCs,
    "nSteps"           : nSteps,
    "Materials"        : Materials,
}

CracKTip = PP(SimulationData)
CracKTip.WriteInputFile()

HDF5 file 'CrackTip_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [12]:
# Create the output file Simul+"_out.hdf5" file
CracKTip.WriteOutputFile(OVERWRITE=True)

In [14]:
WriteXDMF(Simul, elementName, nSteps+1, "Plastic2D")